# Adiabatic SSM model

## Quantifying prediction accuracy

This notebook quantifies the prediction of adiabatic SSM models by doing finite-horizon predictions across the robot's workspace and evaluating the corresponding prediction errors.

In [1]:
from os import listdir
from os.path import join, split
import pickle
import yaml
import numpy as np
from numpy.random import randint
from tqdm.auto import tqdm
import time
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot
from interpolators import InterpolatorFactory

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [4]:
# Adiabatic SSM settings
ROMOrder = 3
INTERPOLATION_METHOD = "nn" #  "idw" # "nn" # "origin_only", "linear", "ct", "nn"
ORIGIN_ONLY = (INTERPOLATION_METHOD == "origin_only")

Trunk settings

In [5]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = join(robot_dir, 'rest_qv.pkl')

In [6]:
# directory containing the SSM models
if ROMOrder == 3:
    model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms"
elif ROMOrder == 1:
    model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_ROMOrder=1"
# model_names = sorted(listdir(model_dir))
model_names = ["northwest", "north", "northeast", "west", "origin", "east", "southwest", "south", "southeast"] # ["north","west", "origin", "east", "south"] # 
print(model_names)
N_models = len(model_names)

['northwest', 'north', 'northeast', 'west', 'origin', 'east', 'southwest', 'south', 'southeast']


Load local SSM models

In [7]:
models = []
observables = "delay-embedding"
for model_name in model_names:
    with open(join(model_dir, model_name, f"SSMmodel_{observables}", "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
V = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B_r = [model['model']['B'] for model in models]
q_bar = [model['model']['q_eq'] for model in models]
u_bar = [model['model']['u_eq'] for model in models]
ROMOrder = models[0]['params']['ROM_order']
SSMOrder = models[0]['params']['SSM_order']
for model in models:
    assert model['params']['ROM_order'] == ROMOrder
    assert model['params']['SSM_order'] == SSMOrder

In [8]:
xy_rest = np.array([q_bar[i][:2] for i in range(N_models)])

Compute observables (delay embedding)

In [9]:
N_DELAY = 4
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Load test trajectory (for now: sum of all trajectories used to regress B matrices)

In [10]:
# test_trajectory_dir = "open-loop"
test_trajectories = []
# for name in tqdm(model_names): # ['origin']: # 
traj_dir = "/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/dataCollection/open-loop_500" # join(model_dir, name, test_trajectory_dir)
(t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, traj_index=0)
y = assemble_observables(z)
test_trajectories.append({
        'name': split(traj_dir)[-1],
        't': t,
        'z': z,
        'u': u,
        'y': y
    })

Combine into one long trajectory

In [11]:
z_tot = np.hstack([traj['z'] for traj in test_trajectories])
y_tot = np.hstack([traj['y'] for traj in test_trajectories])
u_tot = np.hstack([traj['u'] for traj in test_trajectories])
t_tot = np.arange(z_tot.shape[1]) * DT

Interpolate local models to obtain adiabatic SSM model

In [12]:
N_horizon = 5
N_samples = 1000

In [13]:
print(z_tot.shape)
sample_indices = randint(0, z_tot.shape[1], N_samples)

(3, 20001)


In [19]:
interpolation_methods = ["origin_only", "linear", "ct", "nn", "idw", "tps", "qp", "krg", "ls"]
coeff_dict = {
            'w_coeff': w_coeff,
            'V': V,
            'r_coeff': r_coeff,
            'B_r': B_r,
            'u_bar': u_bar,
            'q_bar': q_bar
        }
interpolators = {}
for interpolation_method in interpolation_methods:
    interpolators[interpolation_method] = InterpolatorFactory(interpolation_method, xy_rest, coeff_dict).get_interpolator()

In [20]:
for interpolation_method in tqdm(interpolation_methods, position=0):
    print(f"==================== {interpolation_method} ====================")
    q_samples = []
    rmse_samples = []
    advect_times = []
    for i in tqdm(range(N_samples), position=1):
        start_idx = sample_indices[i]
        end_idx = start_idx + N_horizon
        q_samples.append(z_tot[:3, start_idx])
        # advect ASSM to obtain finite-horizon prediction
        t0 = time.time()
        t, _, y_pred, _, _, _, _ = utils.advect_adiabaticRD_with_inputs(t_tot[start_idx:end_idx], y_tot[:, start_idx], u_tot[:, start_idx:end_idx], y_target=y_tot[:, start_idx:end_idx], interpolator=interpolators[interpolation_method])
        t1 = time.time()
        # compute RMSE
        rmse = np.sum(np.sqrt(np.mean((y_tot[:3, start_idx:end_idx] - y_pred[:3, :])**2, axis=0))) / N_horizon
        rmse_samples.append(rmse)
        advect_times.append(t1 - t0)
    # max_rmse_index = np.argmax(rmse_samples)
    print("max RMSE:", np.nanmax(rmse_samples))
    # print("max RMSE sample idx:", sample_indices[max_rmse_index])
    with open(join(traj_dir, f"{interpolation_method}_rmse_samples.pkl"), "wb") as f:
        pickle.dump(rmse_samples, f)
    with open(join(traj_dir, f"{interpolation_method}_q_samples.pkl"), "wb") as f:
        pickle.dump(q_samples, f)
    with open(join(traj_dir, f"{interpolation_method}_advect_times.pkl"), "wb") as f:
        pickle.dump(advect_times, f)

  0%|          | 0/9 [00:00<?, ?it/s]

==================== origin_only ====================


  0%|          | 0/1000 [00:00<?, ?it/s]

max RMSE: 1.174079487048334
==================== linear ====================


  0%|          | 0/1000 [00:00<?, ?it/s]

...

max RMSE: 0.7963039643882694
==================== krg ====================


  0%|          | 0/1000 [00:00<?, ?it/s]

max RMSE: 0.6660956689214603
==================== ls ====================


  0%|          | 0/1000 [00:00<?, ?it/s]

max RMSE: 0.8676489811212884


In [21]:
plt.close('all')
q_samples = np.stack(q_samples)
rmse_samples = np.array(rmse_samples)
print("q_samples.shape:", q_samples.shape)
print("rmse_samples.shape:", rmse_samples.shape)
plot.prediction_accuracy_map(q_samples, rmse_samples, show=True)

q_samples.shape: (1000, 3)
rmse_samples.shape: (1000,)


Plot prediction accuracy maps for all the different interpolation methods

In [ ]:
plt.close('all')
display_names = {
    "origin_only": "SSMR (origin only)",
    "linear": "ASSMR (barycentric linear)",
    "ct": "ASSMR (Clough-Tocher)",
    "nn": "ASSMR (nearest neighbor)",
    "rbf": "ASSMR (RBF)",
    "idw": "ASSMR (IDW)",
    "tps": "ASSMR (Thin-Plate-Spline)",
    "krg": "ASSMR (Kriging)",
    "qp": "ASSMR (Squared regression)",
    "ls": "ASSMR (Linear regression)"
}
interpolation_methods = ["origin_only", "linear", "ct", "nn", "idw"] # , "tps", "ls", "qp"]

fig, axs = plt.subplots(3, len(interpolation_methods),
                        figsize=(4*len(interpolation_methods), 7),
                        height_ratios=[5, 2, 2],
                        sharey='row', sharex='row', constrained_layout=True)
for i, interpolation_method in enumerate(interpolation_methods):
    with open(join(traj_dir, f"{interpolation_method}_rmse_samples.pkl"), "rb") as f:
        rmse_samples = np.array(pickle.load(f))
    with open(join(traj_dir, f"{interpolation_method}_q_samples.pkl"), "rb") as f:
        q_samples = np.stack(pickle.load(f))
    with open(join(traj_dir, f"{interpolation_method}_advect_times.pkl"), "rb") as f:
        advect_times = np.array(pickle.load(f))
    colorbar = (i == len(interpolation_methods) - 1)
    plot.prediction_accuracy_map(q_samples, rmse_samples, vmax=0.6, ax=axs[0, i], colorbar=colorbar, cax=axs[:, :], show=False)
    plot.boxplot(rmse_samples, ax=axs[1, i], show=False, xlabel="RMSE [mm]", vmax=None)
    plot.boxplot(advect_times, ax=axs[2, i], show=False, xlabel="Advect time [s]", vmax=None)
    axs[0, i].set_title(display_names[interpolation_method])
    # if i > 0:
    #     axs[0, i].set_ylabel("")
# fig.tight_layout()
fig.savefig(join(traj_dir, f"prediction_accuracy.png"), bbox_inches='tight', dpi=200)
fig.show()